In [1]:
from time import sleep
from datetime import date,timedelta
import win32com.client
import pyperclip
import datetime
import math
import subprocess, time
import os, sys
import re
import pandas as pd
import numpy as np
import pyodbc
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine
server = 'bieno-da-d-80166-unilevercom-sql-01.database.windows.net'
database = 'bieno-da-d-80166-unilevercom-sqldb-01'
username = 'PROD_LoadPool'
password = 'Dlk45^8&&dodsslk12Ad7*as'
driver = 'SQL Server Native Client 11.0'
port = '1433'
pd.options.mode.chained_assignment = None
data_directory = 'C:/Users/tristan.dresbach/OneDrive - Unilever/CC_orders/'

In [301]:
#get all the days and dates for the past three months to loop through with SAP:
from datetime import date
from dateutil.relativedelta import relativedelta
today_ = date.today()
one_month = today_ + relativedelta(months=-1)
two_month = today_ + relativedelta(months=-2)
three_month = today_ + relativedelta(months=-3)
start_dates = [three_month,two_month,one_month]
end_dates = [two_month,one_month,today_]
start_dates = [x.strftime('%m/%d/%Y') for x in start_dates]
end_dates = [x.strftime('%m/%d/%Y') for x in end_dates]
file_i = list(range(3))
file_names = []
sales_orgs = ['0001']

In [302]:
#get open sales orders, then plug in numbers in sales order open report

subprocess.Popen(['C:\\Program Files (x86)\\SAP\\FrontEnd\\SapGui\\saplogon.exe'])
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection('P1P - Cordillera - ECC Production', True)

SapGui = win32com.client.GetObject("SAPGUI").GetScriptingEngine
session_id = SapGui.FindById("/app/con[0]").children[0].Id.replace('/app/con[0]/', '')
session = SapGui.FindById(session_id)

def reset_SAP(ses):
    ses.findById("wnd[0]/tbar[0]/btn[12]").press()
    ses.findById("wnd[0]/tbar[0]/btn[12]").press()
    ses.findById("wnd[0]/tbar[0]/btn[3]").press()
    ses.findById("wnd[0]/tbar[0]/btn[3]").press()

#take out sales org loop
for sales_org in sales_orgs:
    for start_d,end_d,i in zip(start_dates,end_dates,file_i):
        print(start_d,end_d,i)
        reset_SAP(session)
        session.findById("wnd[0]/tbar[0]/okcd").text = "VA05"
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/usr/radVBCOM-VBOFF").select()
        session.findById("wnd[0]/usr/ctxtVBCOM-AUDAT").text = start_d
        session.findById("wnd[0]/usr/ctxtVBCOM-AUDAT_BIS").text = end_d
        session.findById("wnd[0]/tbar[1]/btn[33]").press()
        session.findById("wnd[1]/usr/ctxtVBCOM-VKORG").text = sales_org
        session.findById("wnd[1]/usr/ctxtVBCOM-VTWEG").text = "10"
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        session.findById("wnd[0]/tbar[1]/btn[20]").press()
        session.findById("wnd[1]/usr/sub:SAPLKAB1:0400/chkRKAB1-XSUCH[4,0]").selected = True
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        session.findById("wnd[2]/usr/sub:SAPLKAB1:0410/ctxtRKAB1-VONSL[0,23]").text = "ZOR"
        session.findById("wnd[2]/usr/sub:SAPLKAB1:0410/ctxtRKAB1-VONSL[0,23]").caretPosition = 3
        session.findById("wnd[2]/tbar[0]/btn[0]").press()
        session.findById("wnd[0]/usr/radVBCOM-VBOFF").select()
        session.findById("wnd[0]/tbar[0]/btn[0]").press()
        time.sleep(1)
        session.findById("wnd[0]/mbar/menu[0]/menu[4]/menu[2]").select()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\tristan.dresbach\\OneDrive - Unilever\\CC_Orders"
        file_name = 'Open_Order_nb' + str(i) + '_extract.txt'
        session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = file_name
        file_names += [file_name]
        time.sleep(2)
        session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 26
        time.sleep(2)
        session.findById("wnd[1]/tbar[0]/btn[11]").press()
        time.sleep(2)
        
os.system("taskkill /f /im saplogon.exe")

16.11.2020 16.12.2020 0
16.12.2020 16.01.2021 1
16.01.2021 16.02.2021 2


0

In [303]:
#read in the sales orders text files
file_names = ['Open_Order_nb0_extract.txt','Open_Order_nb1_extract.txt','Open_Order_nb2_extract.txt']

sales_docs = []
for file in file_names:
    print(file)
    f = open(data_directory+file,'r')
    text = f.readlines()
    text_mod = [x.replace('\n', '') for x in text]
    text_mod = text_mod[4:]
    del text_mod[-1]
    text_mod = [x.split('|') for x in text_mod]
    text_mod = list(filter(lambda x: len(x) == len(text_mod[0]),text_mod))
    new_mod = []
    for line in text_mod:
        temp_mod = []
        for cell in line:
            temp_mod += [re.sub(' +', ' ',cell).strip()]
        new_mod += [temp_mod]
    sap_df_temp = pd.DataFrame(new_mod[1:],columns=new_mod[0], dtype='object')
    sales_doc = sap_df_temp['Document'].tolist()
    sales_docs += sales_doc

Open_Order_nb0_extract.txt
Open_Order_nb1_extract.txt
Open_Order_nb2_extract.txt


In [9]:
#create  limit dates for SAP data:
from datetime import datetime, date,timedelta
start = date.today() - timedelta(days = 14)
end = date.today() + timedelta(days = 2)
start_ = str(start)[5:7] +'/'+ str(start)[-2:]+'/' + str(start)[0:4]
end_ = str(end)[5:7] +'/'+ str(end)[-2:]+'/' + str(end)[0:4]

now = datetime.now()
window = 3
PGI_start = datetime.today() +timedelta(days=3)
if now.strftime("%A") == 'Friday':
    window = 2
    PGI_start = datetime.today() +timedelta(days=4)
    
PGI_end = PGI_start +timedelta(days=window)
PGI_start = PGI_start.date()
PGI_end = PGI_end.date()
PGI_start_ = str(PGI_start)[5:7] +'/'+ str(PGI_start)[-2:]+'/' + str(PGI_start)[0:4]
PGI_end_ =  str(PGI_end)[5:7] +'/'+ str(PGI_end)[-2:]+'/' +str(end)[0:4]

#copy sales docs:
def to_clipboard(x):
    text = ''
    for item in x:
        text = text + str(item) + '\r\n'
    pyperclip.copy(text)

to_clipboard(sales_docs)

#run SAP to pull line level information based off of the sales order numbers:
COLUMNS_FOR_VARIANT = ['Actual Goods Issue Date','Billing Block','Confirmed Quantity','Customer Name','Customer Purchase Or','Delivery Block','Delivery Block Description','Delivery Note','Distribution Channel','Division','Document Date','OR Crte Tm','Order Quantity','Planned Goods Issue Date','PO Date','PO Type','PoD Creation Date','Sales Order Number','Ship Project','Shipment Number','Shipping Block','Shipping Location','Sold-To Party']


subprocess.Popen(['C:\\Program Files (x86)\\SAP\\FrontEnd\\SapGui\\saplogon.exe'])
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection('P1P - Cordillera - ECC Production', True)

SapGui = win32com.client.GetObject("SAPGUI").GetScriptingEngine
session_id = SapGui.FindById("/app/con[0]").children[0].Id.replace('/app/con[0]/', '')
session = SapGui.FindById(session_id)

#take out sales org=2

#VARIANT UTILIZED REQUIRES THE FOLLOWING COLUMNS:

#load up sap data
#runs sales order report ('Z0OTC0016') list as the first transaction in my favorites:
session.findById("wnd[0]/usr/cntlIMAGE_CONTAINER/shellcont/shell/shellcont[0]/shell").doubleClickNode("F00003")
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_VKGRP-LOW").text = "*"
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_WERKS-LOW").text = "*"  
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_VKORG-LOW").text = "0001"
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_VKORG-LOW").caretPosition = 4
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/btn%_S_VKORG_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "0002"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").caretPosition = 4
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_KTOKD-LOW").text = "*"
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_VKGRP-LOW").text = "*"

#session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_VWADAT-LOW").text = PGI_start_
#session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_VWADAT-HIGH").text = PGI_end_

#the below line opens up the multi-selection for the list of sales order for us to copy paste
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/btn%_S_VBELN_%_APP_%-VALU_PUSH").press()
#the below line pastes them in by clicking the "upload from clipboard button"
session.findById("wnd[1]/tbar[0]/btn[24]").press()
#the below button okays the change and brings us back to the sales order report selection screen
session.findById("wnd[1]/tbar[0]/btn[8]").press()




session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/ctxtS_KTOKD-LOW").text = "*"

session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/btn%_S_ABGRU_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV").select()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,0]").text = "B1"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,0]").caretPosition = 2
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/btn%_S_O_RSN_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV").select()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,0]").text = "Z55"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpNOSV/ssubSCREEN_HEADER:SAPLALDB:3030/tblSAPLALDBSINGLE_E/ctxtRSCSEL_255-SLOW_E[1,0]").caretPosition = 3

session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/chkP_NOSUB").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/chkP_NOSUB").selected = True
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_001/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0100/chkP_DELDC").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_002").select()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_002/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0200/radP_ITEMS").select()
session.findById("wnd[0]/usr/tabsTABSTRIP_MYTABSTRIP/tabpCOMM_002/ssub%_SUBSCREEN_MYTABSTRIP:Z0OTC0718_SALES_ORD_REP:0200/ctxtP_VAR").text = "T_CC"
time.sleep(10)
session.findById("wnd[0]/tbar[1]/btn[8]").press()

#get sap data into text file:
time.sleep(5)
session.findById("wnd[0]/tbar[1]/btn[45]").press()
time.sleep(1)
session.findById("wnd[1]/tbar[0]/btn[0]").press()
time.sleep(1)
session.findById("wnd[1]/usr/ctxtDY_PATH").text = data_directory
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = 'CC_Orders_full.txt'
time.sleep(1)
session.findById("wnd[1]/tbar[0]/btn[11]").press()
time.sleep(1)

os.system("taskkill /f /im saplogon.exe")


first sleep


KeyboardInterrupt: 

In [366]:
#read in all data extracted in the previous cell:
extract_txt = 'CC_Orders_full.txt'
sap_df = pd.DataFrame()
f = open(data_directory+extract_txt,'r')
text = f.readlines()
text_mod = [x.replace('\n', '') for x in text]
text_mod = text_mod[2:]
del text_mod[1]
del text_mod[-1]
text_mod = [x.split('|') for x in text_mod]
new_mod = []
for line in text_mod:
    temp_mod = []
    for cell in line:
        temp_mod += [re.sub(' +', ' ',cell).strip()]
    new_mod += [temp_mod]

#filter out sales orders
sap_df = pd.DataFrame(new_mod[1:],columns=new_mod[0], dtype='object')
sap_df = sap_df[sap_df['Shipping Location'] != '']
sap_df = sap_df[sap_df['Delivery Block'] == '']
sap_df_UCC = sap_df
sap_df = sap_df[sap_df['Material DU'].str.len() < 10].reset_index(drop=True)
sap_df['Material DU'] = [(18-len(x))*'0' + str(x) for x in sap_df['Material DU'] ] 

filter_cols = ['Rejection Reason Description', 'Reason for Rejection',
       'Shipping Block', 'Delivery Block Description', 'Billing Block',
       'Order Item block Description']
for col in filter_cols:
    sap_df = sap_df[sap_df[col] == '']

In [196]:
#Export to input DUs:
all_DUs = list(set(sap_df[sap_df['Confirmed Quantity'] < sap_df['Order Quantity']]['Material DU'].tolist()))
export = pd.DataFrame()
export['MaterialNumber'] = all_DUs
export.to_excel('input_DUs.xlsx')

In [326]:
#if running for subset:
keep_df = pd.read_excel('input_DUs.xlsx')
keep_DUs = keep_df['MaterialNumber'].tolist()
keep_DUs = [(18-len(str(x)))*'0' + str(x) for x in keep_DUs]
sap_df = sap_df[sap_df['Material DU'].isin(keep_DUs)].reset_index()
all_DUs = list(set(sap_df[sap_df['Confirmed Quantity'] < sap_df['Order Quantity']]['Material DU'].tolist()))

In [ ]:
!python Get_Ratio.py

In [327]:
#add plant code to SAP data:
sql =('''select Plant, ShippingPoint as [Shipping Location] from RawReporting_BDL.DimECCShippingPointPerPlant ''')
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
Plants = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

sap_df = pd.merge(sap_df,Plants,how='left',on='Shipping Location')
sap_df['combo'] = sap_df['Material DU'] +'_'+ sap_df['Plant']
#Subset DC of interests in data:
#for DU in DUs_keep:
list_DCs = [list(set(sap_df[sap_df['Material DU']==x]['Plant'])) for x in all_DUs]

In [276]:
#Get APO data, so that we know the on hand position of the DUs:
def reset_SAP(ses):
    ses.findById("wnd[0]/tbar[0]/btn[12]").press()
    ses.findById("wnd[0]/tbar[0]/btn[12]").press()
    ses.findById("wnd[0]/tbar[0]/btn[3]").press()
    ses.findById("wnd[0]/tbar[0]/btn[3]").press()
    

def SAP_load_data(DU,session,UPC):
    #load up Supply planning book for DU
    #this is the 2nd transaction listed in my favorites, transaction ID: /SAPAPO/SDP94
    session.findById("wnd[0]/usr/cntlIMAGE_CONTAINER/shellcont/shell/shellcont[0]/shell").doubleClickNode("F00002")
    session.findById("wnd[0]/shellcont[0]/shell/shellcont[0]/shell/shellcont[0]/shell").pressButton("/SAPAPO/MCPSH_SELECTION_SCREEN")
    session.findById("wnd[1]/usr/cmbGS_GROUP-OBJ").key = "INTERN_MATLO"
    session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/cmbGS_SEL-02-OBJ").key = "9AMATNR"
    if UPC == True:
        session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/txtGS_SEL-02-VAL").text = DU[0]
        print(DU)
        if len(DU) > 1:
            session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/txtGS_SEL-02-VAL").setFocus()
            session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/txtGS_SEL-02-VAL").caretPosition = 8
            session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/btnGS_SEL-02-VAL_MULTI_SEL_ICON").press()
            i = 1
            for item in DU[1:]:
                session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1," + str(i) + "]").text = DU[i]
                i += 1
            session.findById("wnd[2]/tbar[0]/btn[8]").press()
            session.findById("wnd[1]/tbar[0]/btn[8]").press()
        else:
            session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/txtGS_SEL-02-VAL").caretPosition = 8
            session.findById("wnd[1]/tbar[0]/btn[8]").press()
        
    else:
        session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/txtGS_SEL-02-VAL").text = str(DU)  #DU number
        session.findById("wnd[1]/usr/subSEL_SUB_02:/SAPAPO/SAPLMCPSH_SH:0202/txtGS_SEL-02-VAL").caretPosition = 8
        session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/usr/subSUB_MAIN:/SAPAPO/SAPLMSDP_SDP:0027/cntlSDP_TLB_CUSTOM_CONTROL/shellcont/shell").pressButton("MSDP_REFRESH")


def SAP_get_data(DU,DC,session,UPC):
    #get DU/DC data from the book
    exclude = ''
    session.findById("wnd[0]/shellcont[1]/shell/shellcont[0]/shell").pressContextButton("9ALOCNO")
    time.sleep(0.6)
    DC_button = "9ALOCNO-" + str(DC)
    if UPC == True:
        UPC_ = upcs[DU_UPC.index(DU)]
        file_name = 'CC_APO_' + str(UPC_) + '_' + str(DC) + '.txt'
    else: 
        file_name = 'CC_APO_' + str(DU) + '_' + str(DC) + '.txt'
    try:
        session.findById("wnd[0]/shellcont[1]/shell/shellcont[0]/shell").selectContextMenuItem(DC_button)  # DC number
    except:
        pass

    time.sleep(0.6)
    try:
        session.findById("wnd[1]/tbar[0]/btn[71]").press()
        session.findById("wnd[2]/usr/txtRSYSF-STRING").text = str(DC)
        session.findById("wnd[2]/tbar[0]/btn[0]").press()
        session.findById("wnd[3]/usr/lbl[62,2]").setFocus()
        session.findById("wnd[3]/usr/lbl[62,2]").caretPosition = 7
        session.findById("wnd[3]/tbar[0]/btn[2]").press()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
    except:
        pass
    try:
        session.findById("wnd[0]/usr/subSUB_MAIN:/SAPAPO/SAPLMSDP_SDP:0027/cntlSDP_TLB_CUSTOM_CONTROL/shellcont/shell").pressButton("MSDP_DOWNLOAD")
        time.sleep(0.3)
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\tristan.dresbach\\OneDrive - Unilever\\CC_Orders"  # directory
        session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = file_name  # file name
        session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 64
        session.findById("wnd[1]/tbar[0]/btn[11]").press()
    except:
        exclude = file_name

    return exclude
        
    
def SAP_agg_data(DU,session,UPC):
    session.findById("wnd[0]/shellcont[0]/shell/shellcont[1]/shell/shellcont[1]/shell").selectItem("         15","Column1")
    session.findById("wnd[0]/shellcont[0]/shell/shellcont[1]/shell/shellcont[1]/shell").doubleClickItem("         15","Column1")
    session.findById("wnd[0]/usr/subSUB_MAIN:/SAPAPO/SAPLMSDP_SDP:0027/cntlSDP_TLB_CUSTOM_CONTROL/shellcont/shell").pressButton("MSDP_REFRESH")
    session.findById("wnd[0]/usr/subSUB_MAIN:/SAPAPO/SAPLMSDP_SDP:0027/cntlSDP_TLB_CUSTOM_CONTROL/shellcont/shell").pressButton("MSDP_DOWNLOAD")
    session.findById("wnd[1]/tbar[0]/btn[0]").press()
    session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\tristan.dresbach\\OneDrive - Unilever\\CC_Orders"  #directory
    if UPC == True:
        UPC_ = upcs[DU_UPC.index(DU)]
        file_name = 'Full_Data' +'_' + str(UPC_) + '.txt'
    else: 
        file_name = 'Full_Data' +'_'+ str(DU) + '.txt'
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = file_name  #file name
    session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 64
    session.findById("wnd[1]/tbar[0]/btn[11]").press()

SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
SAPConnection = application.Children(0)
session = SAPConnection.Children(0)

exclusions = []
for DU in all_DUs:
    print(DU)
    reset_SAP(session)
    SAP_load_data(DU,session,False)
    nb = all_DUs.index(DU)
    for DC in list_DCs[nb]: # DC_UPC[nb]:
        exclusions += [SAP_get_data(DU,DC,session,False)]

000000000067549924
000000000067736682
000000000068516275
000000000084139090
000000000068309313
000000000067146355
000000000067723726
000000000067309864
000000000067789313
000000000067881630
000000000067353515
000000000067545847
000000000067794169
000000000067733288
000000000068117574
000000000067278447
000000000068518500
000000000067726290
000000000067872666
000000000067805573
000000000084143851
000000000067895460
000000000067726286
000000000068117572
000000000067549928
000000000067291237
000000000068410355
000000000067751794
000000000067718868
000000000068289143
000000000084138392


In [328]:
from datetime import datetime
#read in the APO text files we just pulled and convert them into a df:
def get_information(directory):
    file_list = []
    for i in os.listdir(directory):
        a = os.stat(os.path.join(directory,i))
        file_list.append([i,time.ctime(a.st_atime),time.ctime(a.st_ctime)]) #[file,most_recent_access,created]
    return file_list

full_files = get_information("C:/Users/tristan.dresbach/OneDrive - Unilever/CC_Orders")
file_names = [x[0] for x in full_files]
keep_files = [x for x in file_names if x[:6] == 'CC_APO']
#MODIFY THIS FOR ONLY TODAY's RUN APO DATA
keep_files = [x for x in keep_files if x.split('_')[2] in all_DUs]

file_names = []
names = []
all_data = {}
exclusions = []

for file_n in keep_files:
    temp_ = []
    try:
        with open(file_n) as f:
            lines_after_6 = f.readlines()[6:]
            for line in lines_after_6:
                line = line.replace('"',"").split(",")
                line.remove('\n')
                temp_.append(line)
                    
        df_temp = pd.DataFrame(temp_[1:],columns = temp_[0])
        
        if file_n[0] in [str(nb) for nb in list(range(6))]:
            file_n = file_n[1:]
            
        name_ = file_n[:-4]
        all_data.update({name_:df_temp})
        names.append(name_)
    except:
        exclusions.append(file_n)

for df_name in all_data:
    all_data[df_name] = all_data[df_name].fillna(0)
    all_data[df_name] = all_data[df_name].iloc[:,:13]

#create combos to find receiving Inventory:
combos = [x.split('_')[2] +'_'+ x.split('_')[3] for x in names]
combos_q = []
date_q = []
q_q = []

for combo in combos:
    temp_df = all_data[('CC_APO_' + combo)]
    temp_df = temp_df[temp_df['SNP PLAN'] == 'Total Receipts']
    temp_df = temp_df.iloc[:,3:]
    for day in temp_df:
        check_nb = list(temp_df[day])[0]
        if len(check_nb) > 1:
            combos_q += [combo]
            date_q += [day]
            q_q += [int(check_nb)]

STOs = pd.DataFrame()
STOs['combo'] = combos_q
STOs['day'] = date_q
STOs['quantity'] = q_q

orders = sap_df[['Sales Order Number','Material DU','Order Quantity','Planned Goods Issue Date','Plant','combo','PO Date','Sold-To Party']]

def rep1(x):
    try:
        if x == '0,000':
            return 0
        else:
            return float(str(x).replace('.','').replace(',','.').replace('-',''))
    except:
        return 0
orders['Order Quantity'] = [rep1(x) for x in orders['Order Quantity']]
orders['Planned Goods Issue Date'] = [datetime.strptime(x, '%m/%d/%Y') for x in orders['Planned Goods Issue Date']]
orders['PO Date'] = [datetime.strptime(x, '%m/%d/%Y') for x in orders['PO Date']]
STOs['day'] = [datetime.strptime(x, '%m/%d/%Y') for x in STOs['day']]

In [329]:
now = datetime.now()
#what day window will we allow oursels to check for sales orders, STOs and OH
window = 3
OH_col = (datetime.today() +timedelta(days=3)).strftime("%m/%d/%Y")
if now.strftime("%A") == 'Friday':
    window = 2
    OH_col = (datetime.today() +timedelta(days=4)).strftime("%m/%d/%Y")
    
combos_t = []
date_t = []
OH_q = []
OH_q_alt = []

def conv_str(item):
    if item == '':
        item = 0
    else:
        item = int(item)
    return item
#loop through each combo and calculate based off of receipts and demand what our OH inventory is
for combo in combos:
    temp_df = all_data[('CC_APO_' + combo)]
    temp_df_sub = temp_df[temp_df['SNP PLAN'] == 'Stock on Hand (Proj)']
    OH_nb = list(temp_df_sub[OH_col])[0]
    OH_nb = conv_str(OH_nb)
    if OH_nb < 0:
        OH_nb = 0
    combos_t += [combo]
    date_t += [OH_col]
    OH_q += [OH_nb]
    
    OH_recalc = 0
    OH_temp = 0
    
    nb_end = temp_df.columns.tolist().index(OH_col)
    OH_temp = temp_df[temp_df['SNP PLAN'] == 'Stock on Hand (Proj)']['Initial'].tolist()[0]
    OH_temp = conv_str(OH_temp)
    for i in range(3,nb_end+1):
        OH_recalc = OH_temp
        receipts = temp_df[temp_df['SNP PLAN'] == 'Total Receipts'][temp_df.columns.tolist()[i]].tolist()[0]
        demand = temp_df[temp_df['SNP PLAN'] == 'Total Demand'][temp_df.columns.tolist()[i]].tolist()[0]
        demand = conv_str(demand)
        receipts = conv_str(receipts)
        OH_recalc += receipts
        OH_recalc -= demand
        if OH_recalc < 0:
            OH_temp = 0
        else:
            OH_temp = OH_recalc
    
    OH_q_alt += [OH_temp]

OHs = pd.DataFrame()
OHs['combo'] = combos_t
OHs['date'] = date_t
OHs['alt_quantity'] = OH_q
OHs['day'] = [datetime.strptime(x, '%m/%d/%Y') for x in OHs['date']]
OHs['quantity'] = OH_q_alt

try:
    OHs['day'] = OHs['day'].apply(lambda x: x.date())
except:
    pass

In [331]:
#bring in ratios, generated previously:
ratio_data = pd.read_excel('ratio_data.xlsx').reset_index(drop=True)
#ratios = pd.read_excel('ratios.xlsx')
orders_day = orders.copy()
orders_day['key'] = orders_day['Material DU'] +'_' + orders_day['Sold-To Party']
ratios = ratio_data.copy()
#ratios['ratio'] = [ 0 if x == 'no loss' else x for x in ratios['ratio']]
ratios['MaterialNumber'] = [str(int(list(x.split('_'))[0])) for x in ratios['key']]
ratios['MaterialNumber'] = [ (18-len(str(x)))*'0'+str(x) for x in ratios['MaterialNumber']]
ratios['Sold-To Party'] = [str(int(list(x.split('_'))[1])) for x in ratios['key']]
ratios['key'] = ratios['MaterialNumber'] +'_' + ratios['Sold-To Party']
ratios_m = ratios[['key','ratio']]
orders_r = pd.merge(orders_day,ratios_m,how='left',on='key')
orders_r['Assigned_q'] = [0 for x in range(len(orders_r))]
orders_r['Planned Goods Issue Date'] = orders_r['Planned Goods Issue Date'].apply(lambda x: x.date())
orders_r['Assigned_key'] = orders_r['Sales Order Number'] +'_'+ orders_r['Material DU']
orders_r_frozen = orders_r.copy()

In [332]:
orders_r = orders_r.fillna(0)
orders_r['ratio'] = [ 0 if x == 'no loss' else x for x in orders_r['ratio']]

In [333]:
#MODIFY THIS LATER:
try:
    STOs['day'] = STOs['day'].apply(lambda x: x.date())
except:
    pass

STOs['key'] = STOs['combo'] +'_'+ [str(x) for x in STOs['day']] +'_'+ [str(x) for x in STOs['quantity']]

In [334]:
#get all layer multiples:
DU_lay = list(set(orders_r['Material DU']))

sql ='''
select MaterialNumber,Denominator,Numerator from RawReporting_BDL.DimMaterialUnitOfMeasure where UnitOfMeasureCode = 'LAY'
and MaterialNumber in '''  + str(DU_lay).replace('[','(').replace(']',')')

#EDIT BACK SALES ORGS: '0002','0003','0007'
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
lay_df = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [335]:
#loop through combos, OH first:
all_assigned_keys = []
all_assigned_q_OH = []
all_assigned_stos = []
all_assigned_orders = []
all_assigned_q_STO = []

#essentially, we are trying to allocate inventory to a sales order line based off the ratio. TO do so, we need to know
#our inventory window during the given 2 or 3 day time period. Once we know how much we have when, we can determine how to modify
#the existing sales orders based off of our ratio
for combo in combos: #in combos:
    print(combo)
    layer = list(lay_df[lay_df['MaterialNumber'] == combo.split('_')[0]]['Numerator'])[0]
    print(layer)
    temp_orders = orders_r[orders_r['combo'] == combo]
    temp_orders = temp_orders[(temp_orders['Planned Goods Issue Date'] >= list(OHs['day'])[0]) & (temp_orders['Planned Goods Issue Date'] <= (list(OHs['day'])[0] + timedelta(days=window)))]
    temp_orders = temp_orders.sort_values(['ratio'],ascending=False).reset_index(drop=True)
    temp_orders = temp_orders.sort_values(['ratio', 'Planned Goods Issue Date','Order Quantity'], ascending=[False, True,False]).reset_index(drop=True)
    OH = list(OHs[OHs['combo']==combo]['quantity'])[0]
    print('base OH: ',OH)
    i = 0
    while (OH > 0  and i < len(temp_orders)):
        q_ord = temp_orders['Order Quantity'][i]
        #OH_if_neg = math.floor(OH/layer)*layer
        OH_if_neg = OH
        OH = OH - q_ord
        assigned_key = temp_orders['Assigned_key'][i]
        row = list(orders_r['Assigned_key']).index(assigned_key)
        if OH < 0:
            all_assigned_keys += [assigned_key]
            temp_orders.at[i,'Assigned_q'] = OH_if_neg
            orders_r.at[row,'Assigned_q'] = OH_if_neg
            all_assigned_q_OH += [OH_if_neg]
        else:
            orders_r.at[row,'Assigned_q'] = q_ord
            temp_orders.at[i,'Assigned_q'] = q_ord
            all_assigned_keys += [assigned_key]
            all_assigned_q_OH += [q_ord]
            
        i += 1
        
    temp_stos = STOs[STOs['combo'] == combo]
    temp_stos = temp_stos[(temp_stos['day'] > list(OHs['day'])[0]) & (temp_stos['day'] <= (list(OHs['day'])[0] + timedelta(days=window)))]
    temp_stos = temp_stos.sort_values(['day']).reset_index(drop=True)
    
    for n in range(len(temp_stos)):
        day = temp_stos['day'][n]
        temp_sto_orders = temp_orders[temp_orders['Planned Goods Issue Date'] >= day].reset_index(drop=True)
        STO_q = temp_stos['quantity'][n]
        sto_key = temp_stos['key'][n]
        po_n = 0
        while( STO_q > 0 and po_n < len(temp_sto_orders)):
            order_q =  temp_sto_orders['Order Quantity'][po_n]
            assig_q = temp_sto_orders['Assigned_q'][po_n]
            assigned_key = temp_sto_orders['Assigned_key'][po_n]
            row_t = list(temp_orders['Assigned_key']).index(assigned_key)
            row = list(orders_r['Assigned_key']).index(assigned_key)
            print(STO_q,order_q,assig_q)
            
            if order_q > assig_q:
                print('here',order_q,assig_q)
                #rem_q = order_q - math.floor(assig_q/layer)*layer
                rem_q = order_q - assig_q
                STO_q_if_neg = STO_q
                STO_q = STO_q - rem_q
                print(STO_q,rem_q,assig_q)
                
                if STO_q < 0:
                    temp_orders.at[row_t,'Assigned_q'] += STO_q_if_neg
                    temp_sto_orders.at[po_n,'Assigned_q'] += STO_q_if_neg
                    orders_r.at[row,'Assigned_q'] += STO_q_if_neg
                    all_assigned_stos += [sto_key]
                    all_assigned_orders += [assigned_key]
                    all_assigned_q_STO += [STO_q_if_neg]

                else:
                    temp_orders.at[row_t,'Assigned_q'] += rem_q
                    temp_sto_orders.at[po_n,'Assigned_q'] += rem_q
                    orders_r.at[row,'Assigned_q'] += rem_q
                    all_assigned_stos += [sto_key]
                    all_assigned_orders += [assigned_key]
                    all_assigned_q_STO += [rem_q]
            
            po_n +=1 
            print(STO_q,po_n)
  
    #for STO in STOs_temp:
    #now for STOs:
    
df_mod = orders_r[orders_r['combo'].isin(combos)].reset_index(drop=True)

000000000067146355_1419
25.0
base OH:  927
000000000067146355_2300
25.0
base OH:  3474
000000000067146355_2400
25.0
base OH:  6844
000000000067146355_3989
25.0
base OH:  9286
000000000067278447_1419
15.0
base OH:  333
000000000067278447_2300
15.0
base OH:  0
000000000067278447_2400
15.0
base OH:  0
000000000067278447_3989
15.0
base OH:  341
000000000067291237_2400
15.0
base OH:  0
000000000067291237_3989
15.0
base OH:  0
000000000067309864_1419
11.0
base OH:  737
616 33.0 33
616 1
616 154.0 154
616 2
616 154.0 154
616 3
000000000067309864_2400
11.0
base OH:  0
462 22.0 0
here 22.0 0
440.0 22.0 0
440.0 1
000000000067309864_3989
11.0
base OH:  16052
000000000067353515_1419
17.0
base OH:  1530
2040 170.0 170
2040 1
2040 340.0 340
2040 2
2040 1020.0 1020
2040 3
2040 170.0 0
here 170.0 0
1870.0 170.0 0
1870.0 4
1870.0 170.0 0
here 170.0 0
1700.0 170.0 0
1700.0 5
1700.0 170.0 0
here 170.0 0
1530.0 170.0 0
1530.0 6
1530.0 170.0 0
here 170.0 0
1360.0 170.0 0
1360.0 7
1360.0 170.0 0
here 170.0 

21590 68.0 68
21590 1
21590 68.0 68
21590 2
21590 17.0 17
21590 3
000000000067789313_1419
15.0
base OH:  11172
2640 15.0 15
2640 1
2640 45.0 45
2640 2
2640 240.0 240
2640 3
2640 120.0 120
2640 4
2640 120.0 120
2640 5
000000000067789313_2300
15.0
base OH:  0
3600 45.0 0
here 45.0 0
3555.0 45.0 0
3555.0 1
3555.0 120.0 0
here 120.0 0
3435.0 120.0 0
3435.0 2
3435.0 45.0 0
here 45.0 0
3390.0 45.0 0
3390.0 3
3390.0 240.0 0
here 240.0 0
3150.0 240.0 0
3150.0 4
3150.0 15.0 0
here 15.0 0
3135.0 15.0 0
3135.0 5
1440 45.0 45
1440 1
1440 120.0 120
1440 2
1440 45.0 45
1440 3
1440 240.0 240
1440 4
1440 15.0 15
1440 5
000000000067789313_2400
15.0
base OH:  79
9120 210.0 79
here 210.0 79
8989.0 131.0 79
8989.0 1
8989.0 240.0 0
here 240.0 0
8749.0 240.0 0
8749.0 2
000000000067789313_3989
15.0
base OH:  0
6480 735.0 0
here 735.0 0
5745.0 735.0 0
5745.0 1
5745.0 240.0 0
here 240.0 0
5505.0 240.0 0
5505.0 2
5505.0 120.0 0
here 120.0 0
5385.0 120.0 0
5385.0 3
5385.0 120.0 0
here 120.0 0
5265.0 120.0 0
5265

12.0
base OH:  1382
000000000067895460_3989
12.0
base OH:  419
4092 48.0 48
4092 1
4092 72.0 72
4092 2
4092 24.0 24
4092 3
4092 24.0 24
4092 4
4092 24.0 24
4092 5
4092 48.0 48
4092 6
4092 132.0 0
here 132.0 0
3960.0 132.0 0
3960.0 7
3960.0 132.0 0
here 132.0 0
3828.0 132.0 0
3828.0 8
3828.0 132.0 0
here 132.0 0
3696.0 132.0 0
3696.0 9
000000000068117572_1419
13.0
base OH:  4813
1716 143.0 143
1716 1
1716 143.0 143
1716 2
1716 143.0 143
1716 3
1716 143.0 143
1716 4
1716 65.0 65
1716 5
1716 39.0 39
1716 6
1716 13.0 13
1716 7
1716 286.0 286
1716 8
000000000068117572_2300
13.0
base OH:  6296
000000000068117572_2400
13.0
base OH:  6306
000000000068117572_3989
13.0
base OH:  15569
000000000068117574_1419
13.0
base OH:  1244
000000000068117574_2300
13.0
base OH:  807
000000000068117574_2400
13.0
base OH:  4117
000000000068117574_3989
13.0
base OH:  5411
429 143.0 143
429 1
429 143.0 143
429 2
429 143.0 143
429 3
429 143.0 143
429 4
429 143.0 143
429 5
429 286.0 286
429 6
429 78.0 78
429 7
429

In [336]:
#get all layer multiples (some items are only sold in certain multiples, ie. an ice cream item might be sold in multiple of 12):
DU_mod = list(set(df_mod['Material DU']))

sql ='''
select MaterialNumber as 'Material DU',Denominator,Numerator,B.SubDivision2Name,B.CategoryName from RawReporting_BDL.DimMaterialUnitOfMeasure A
left join (select SKUCode,SubDivision2Name,CategoryName from [RawReporting_BDL].[HierarchyGlobalProduct]) B
on A.MaterialNumber = B.SKUCode 
where UnitOfMeasureCode = 'LAY'

and MaterialNumber in '''  + str(DU_mod).replace('[','(').replace(']',')')

connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
lay_df = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [337]:
#get only lines that need to be changed and check multiple of quantity change (some items can only be purchased/sent in certain multiples, ie. a multiple of 15 item can't be decreased by 12 cases):
df_mod = pd.merge(df_mod,lay_df,how='left',on='Material DU')
change_df = df_mod[df_mod['Order Quantity'] != df_mod['Assigned_q']]
change_df['Adj_assigned_Q'] = [math.floor(q/layer_q)*layer_q if type_ == 'Ice Cream' else q  for q,layer_q,type_ in zip(change_df['Assigned_q'],change_df['Numerator'],change_df['CategoryName'])]
change_df['Action'] = ['Reject line' if x == 0 else 'Decrease by: '+str(y-x) for x,y in zip(change_df['Adj_assigned_Q'],change_df['Order Quantity'])]
output_df = change_df[['Sales Order Number', 'Material DU', 'Order Quantity',
       'Planned Goods Issue Date', 'Plant', 'PO Date',
       'Sold-To Party', 'ratio', 'Numerator', 'CategoryName',
       'Adj_assigned_Q', 'Action']]

In [367]:
#merge the dfs to get full information needed:
#this is the file that genpact or a script will utilize to be able to know which sales order lines to modify
sap_df_UCC = sap_df_UCC[sap_df_UCC['Material DU'].str.len() > 10].reset_index(drop=True)
sap_df_UCC['Order Quantity'] = [rep1(x) for x in sap_df_UCC['Order Quantity']]
sap_df_UCC['PO Date'] = [datetime.strptime(x, '%m/%d/%Y') for x in sap_df_UCC['PO Date']]
sap_df_UCC.rename(columns={'Line Item':'Line Item -2','Material DU':'UCC 14'},inplace=True)
sap_df_UCC['Line Item'] = [str(int(x)+2) for x in sap_df_UCC['Line Item -2']]

cols_UCC = ['Sales Order Number', 'Sold-To Party','Line Item', 'Customer Name']
merge_UCC = sap_df_UCC[cols_UCC+['UCC 14']]

merge_sap_df  = sap_df[['Sales Order Number','Material DU','Sold-To Party','Line Item','Customer Name']]
merged_sap_df = pd.merge(output_df,merge_sap_df,how='left',on=['Sales Order Number','Material DU','Sold-To Party'])
full_output_df = pd.merge(merged_sap_df,merge_UCC,how='left',on=cols_UCC)

#reconfigure line items numbers:
full_output_df.rename(columns={'Line Item':'Line Item DU'},inplace=True)
full_output_df['Line Item UCC'] = [str(int(x)-2) for x in full_output_df['Line Item DU']]